In [1]:
# app.py
import streamlit as st
import numpy as np
import joblib
import re
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords

In [2]:
# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Load trained model and vectorizer
model = joblib.load("fake_news_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

c:\Users\DELL\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\DELL\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\DELL\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.7.0 when using version 1.5.1. This might l

In [4]:
# Function to clean input text (same as training)
def clean_text(text):
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [5]:
# Streamlit App UI
st.set_page_config(page_title="Fake News Detector")
st.title("📰 Fake News Detector")
st.markdown("Type or paste a news **headline or short article** below to check if it's **real or fake**.")

2025-07-09 17:12:47.694 
  command:

    streamlit run c:\Users\DELL\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [6]:
user_input = st.text_area("🖊️ Enter News Text Here")

2025-07-09 17:12:47.752 Session state does not function when running a script without `streamlit run`


In [7]:
if st.button("Analyze"):
    if user_input.strip() == "":
        st.warning("⚠️ Please enter some text to analyze.")
    else:
        cleaned = clean_text(user_input)
        polarity = TextBlob(cleaned).sentiment.polarity

        input_vec = tfidf.transform([cleaned])
        full_input = np.hstack([input_vec.toarray(), [[polarity]]])

        prediction = model.predict(full_input)[0]
        confidence = round(np.max(model.predict_proba(full_input)) * 100, 2)

        st.subheader("🧠 Prediction Result:")
        if prediction == 0:
            st.success(f"✅ This news is **REAL** ({confidence}% confidence)")
        else:
            st.error(f"❌ This news is **FAKE** ({confidence}% confidence)")